# Capstone Project

## Finger Detection and Counting

## Imports

In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise


### Global Variables

We will use these as we go along.

In [2]:
background = None

accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

## Finding Average Background Value

The function calculates the weighted sum of the input image src and the accumulator dst so that dst becomes a running average of a frame sequence:

In [3]:
def calc_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)
        

## Segment the Hand Region in Frame

In [4]:
def segment(frame, threshold_min=25):
    
    diff = cv2.absdiff(background.astype('uint8'), frame)
    
    ret, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    image, contours , hierarchy - cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours ) == 0:
        return None
    else:
         # ASSUMING THE LARGEST EXTERNAL CONTOUR IN ROI, IS THE HAND
        hand_segment = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment)
        

## Counting Fingers with a Convex Hull

We just calculated the external contour of the hand. Now using that segmented hand, let's see how to calculate fingers. Then we can count how many are up!

Example of ConvexHulls:

In [5]:
def count_fingers(thresholded,hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    # TOP
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()[0]])
    bottom =  tuple(conv_hull[conv_hull[:,:,1].argmax()[0]])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()[0]])
    right =  tuple(conv_hull[conv_hull[:,:,0].argmax()[0]])
    
    cX = (left[0] + right[0]) // 2
    cy = (left[1] + right[1]) // 2
    
    distance = pairwise.euclidean_distances([cX, cY], Y=[left, right, top, bottom])[0]
    
    max_distance= distance.max()
    
    
    # Create a circle with 90% radius of the max euclidean distance
    radius = int(0.9* max_distance)
    
    circumfrence = (2*np.pi*radius)
    
    # Not grab an ROI of only that circle
    circular_roi = np.zeros(thresholded[:2], dtype='uint8')
    
     # draw the circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    # Using bit-wise AND with the cirle ROI as a mask.
    # This then returns the cut out obtained using the mask on the thresholded hand image.
    circular_roi = cv2.bitwise_and(thresholded,thresholded, mask=circular_roi)
    
    # Grab contours in circle ROI
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in coutours:
        (x,y,w,h)  = cv2.boundingRect(cnt)
        
        # Increment count of fingers based on two conditions:
        
        # 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist =  (cY  + (cY*0.25)) > (y+h)
        
         # 2. Number of points along the contour does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points off the hand)
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count +=1
            
    return count

## Run Program

In [ ]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    ret, frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    gray =  cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray ,accumulated_weight )
        if num_frames < 59:
            cv2.putText(frame_copy, 'WAIT. GETTING BACKGROUNG' , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 1 , (0,0,255), 2)
            cv2.imshow('Finger Count', frame_copy)
        else:
            hand = segment(gray)
            
            if hand is not None:
                
                thresholded, hand_segment = hand
                
                # DRAWS CONTOURS AROUND REAL HAND IN LIVE STREAM
                cv2.drawContours(frame_copy, [hand_segment+(roi_right,roi_top)] ,-1,(255,0,0),5)
                
                fingers = count_fingers(thresholded, hand_segment)
                
                cv2.putText(frame_copy, str(fingers), (70,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
                
                cv2.imshow('Thresholded',thresholded)
        cv2.rectangle(frame_copy, (roi_left,roi_top), (roi_right,roi_bottom), (0,0,255),5)
        
        cv2.imshow('Finger Count', frame_copy)
        
        k = cv2.waitKey(1) & 0xFF
    
        if k == 27:
            break
        
cam.release()
cv2.destroyAllWindows()
                
    
    